In [4]:
import csv
import os
import cv2
import random

In [5]:
anno_paths_bulb = []
anno_paths_box = []
img_dirs = []
anno_base_path = r'NEW_IMAGES/Annotations/Annotations'
img_base_path = r'NEW_IMAGES'

for train_type in ['dayTrain', 'nightTrain']: 
    anno_train_path = os.path.join(anno_base_path, train_type)
    img_train_path = os.path.join(img_base_path, train_type, train_type)

    for clip in os.listdir(anno_train_path):
        clip_path = os.path.join(anno_train_path, clip)
        
        if os.path.isdir(clip_path):
            anno_file = os.path.join(clip_path, 'frameAnnotationsBOX.csv')
            if os.path.isfile(anno_file):
                anno_paths_box.append(anno_file)

        if os.path.isdir(clip_path):
            anno_file = os.path.join(clip_path, 'frameAnnotationsBULB.csv')
            if os.path.isfile(anno_file):
                anno_paths_bulb.append(anno_file)

    for clip in os.listdir(img_train_path):
        clip_path = os.path.join(img_train_path, clip)
        
        if os.path.isdir(clip_path):
            img_dir = os.path.join(clip_path, 'frames')
            if os.path.isdir(img_dir):
                img_dirs.append(img_dir)

print('img_dirs :',img_dirs)
print()
print('anno_path_bulb_dir :',anno_paths_bulb)
print()
print('anno_path_box :',anno_paths_box)

img_dirs : ['NEW_IMAGES\\dayTrain\\dayTrain\\dayClip1\\frames', 'NEW_IMAGES\\dayTrain\\dayTrain\\dayClip10\\frames', 'NEW_IMAGES\\dayTrain\\dayTrain\\dayClip11\\frames', 'NEW_IMAGES\\dayTrain\\dayTrain\\dayClip12\\frames', 'NEW_IMAGES\\dayTrain\\dayTrain\\dayClip13\\frames', 'NEW_IMAGES\\dayTrain\\dayTrain\\dayClip2\\frames', 'NEW_IMAGES\\dayTrain\\dayTrain\\dayClip3\\frames', 'NEW_IMAGES\\dayTrain\\dayTrain\\dayClip4\\frames', 'NEW_IMAGES\\dayTrain\\dayTrain\\dayClip5\\frames', 'NEW_IMAGES\\dayTrain\\dayTrain\\dayClip6\\frames', 'NEW_IMAGES\\dayTrain\\dayTrain\\dayClip7\\frames', 'NEW_IMAGES\\dayTrain\\dayTrain\\dayClip8\\frames', 'NEW_IMAGES\\dayTrain\\dayTrain\\dayClip9\\frames', 'NEW_IMAGES\\nightTrain\\nightTrain\\nightClip1\\frames', 'NEW_IMAGES\\nightTrain\\nightTrain\\nightClip2\\frames', 'NEW_IMAGES\\nightTrain\\nightTrain\\nightClip3\\frames', 'NEW_IMAGES\\nightTrain\\nightTrain\\nightClip4\\frames', 'NEW_IMAGES\\nightTrain\\nightTrain\\nightClip5\\frames']

anno_path_bulb_di

In [6]:
anno_paths_box = anno_paths_box
anno_paths_bulb = anno_paths_bulb
img_dirs = img_dirs

print("Box Annotation Paths:", anno_paths_box)
print("\nBulb Annotation Paths:", anno_paths_bulb)
print("\nImage Directories:", img_dirs)


Box Annotation Paths: ['NEW_IMAGES/Annotations/Annotations\\dayTrain\\dayClip1\\frameAnnotationsBOX.csv', 'NEW_IMAGES/Annotations/Annotations\\dayTrain\\dayClip10\\frameAnnotationsBOX.csv', 'NEW_IMAGES/Annotations/Annotations\\dayTrain\\dayClip11\\frameAnnotationsBOX.csv', 'NEW_IMAGES/Annotations/Annotations\\dayTrain\\dayClip12\\frameAnnotationsBOX.csv', 'NEW_IMAGES/Annotations/Annotations\\dayTrain\\dayClip13\\frameAnnotationsBOX.csv', 'NEW_IMAGES/Annotations/Annotations\\dayTrain\\dayClip2\\frameAnnotationsBOX.csv', 'NEW_IMAGES/Annotations/Annotations\\dayTrain\\dayClip3\\frameAnnotationsBOX.csv', 'NEW_IMAGES/Annotations/Annotations\\dayTrain\\dayClip4\\frameAnnotationsBOX.csv', 'NEW_IMAGES/Annotations/Annotations\\dayTrain\\dayClip5\\frameAnnotationsBOX.csv', 'NEW_IMAGES/Annotations/Annotations\\dayTrain\\dayClip6\\frameAnnotationsBOX.csv', 'NEW_IMAGES/Annotations/Annotations\\dayTrain\\dayClip7\\frameAnnotationsBOX.csv', 'NEW_IMAGES/Annotations/Annotations\\dayTrain\\dayClip8\\fra

In [12]:
def read_annotaion(csv_file_path):
    bounding_boxes = []
    with open(csv_file_path, mode='r') as file:
        csv_reader = csv.reader(file, delimiter=';')
        next(csv_reader)
        for row in csv_reader:
            filename = row[0]
            annotation_tag = row[1]
            upper_left_x = int(row[2])
            upper_left_y = int(row[3])
            lower_right_x = int(row[4])
            lower_right_y = int(row[5])
            bounding_boxes.append((filename, annotation_tag, (upper_left_x, upper_left_y, lower_right_x, lower_right_y)))
    return bounding_boxes

In [13]:
box_output_dir = 'NEW_IMAGES/cropped_boxes'
bulb_output_dir = 'NEW_IMAGES/cropped_bulbs'


os.makedirs(box_output_dir, exist_ok=True)
os.makedirs(bulb_output_dir, exist_ok=True)

images = []


for csv_file_box, csv_file_bulb, img_dir in zip(anno_paths_box, anno_paths_bulb, img_dirs):
    box_annotations = read_annotaion(csv_file_box)
    bulb_annotations = read_annotation(csv_file_bulb)
    
    
    image_files = sorted([f for f in os.listdir(img_dir) if f.endswith(('.jpg'))])
    selected_images = image_files

    # Collect images and corresponding annotations in the array
    for filename in selected_images:
        img_path = os.path.join(img_dir, filename)

        # Check if image file exists
        if os.path.isfile(img_path):
            original_img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)

            # Initialize crops as None
            box_crop, bulb_crop = None, None

            # Process box annotation
            for box in box_annotations:
                if os.path.basename(box[0]) == filename:
                    x1, y1, x2, y2 = box[2]
                    box_crop = original_img[y1:y2, x1:x2]
                    # Save box image
                    box_output_path = os.path.join(box_output_dir, f"box_{filename}")
                    cv2.imwrite(box_output_path, cv2.cvtColor(box_crop, cv2.COLOR_RGB2BGR))
                    break

            # Process bulb annotation
            for bulb in bulb_annotations:
                if os.path.basename(bulb[0]) == filename:
                    x1, y1, x2, y2 = bulb[2]
                    bulb_crop = original_img[y1:y2, x1:x2]
                    # Save bulb image
                    bulb_output_path = os.path.join(bulb_output_dir, f"bulb_{filename}")
                    cv2.imwrite(bulb_output_path, cv2.cvtColor(bulb_crop, cv2.COLOR_RGB2BGR))
                    break

            images.append((original_img, box_crop, bulb_crop))



KeyboardInterrupt: 